<a href="https://colab.research.google.com/github/RockZeroAxl/Tsuki/blob/main/NLP_Parsing_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import unicodedata
from datetime import datetime

class TextCleaner:
    """
    處理醫療文本的基礎清洗工具
    Demonstrates: String manipulation, Regex, Unicode normalization
    """

    @staticmethod
    def normalize_text(text: str) -> str:
        """
        將文本進行正規化：
        1. 轉為半形 (NFKC)
        2. 清除 HTML tag
        3. 統一空白字符
        """
        if not text:
            return ""

        # Unicode 正規化 (解決全形數字、特殊符號問題)
        text = unicodedata.normalize('NFKC', text)

        # 移除 XML/HTML tags
        text = re.sub(r'</?\w+>', ' ', text)

        # 移除特殊干擾字元 (依據專案經驗)
        text = re.sub(r'[■□★※＊$~\?]+', '', text)

        # 將多餘的空白與換行壓縮為單一空白，但保留段落結構可視需求調整
        text = re.sub(r'\s+', ' ', text).strip()

        return text

class DateParser:
    """
    處理台灣醫療報告中常見的日期格式 (民國/西元混合)
    Demonstrates: Datetime logic, Error handling
    """

    @staticmethod
    def parse_report_date(date_str: str) -> str | None:
        """
        支援格式：
        - 20230101 (YYYYMMDD)
        - 1120101 (ROC YYYMMDD)
        - 2023-01-01
        - 2023/01/01
        """
        if not date_str:
            return None

        date_str = date_str.strip()

        # 模式 A: YYYY-MM-DD or YYYY/MM/DD
        if re.match(r"^\d{4}[-/]\d{2}[-/]\d{2}$", date_str):
            return date_str.replace('/', '-')

        # 模式 B: YYYYMMDD (8碼)
        if date_str.isdigit() and len(date_str) == 8:
            return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"

        # 模式 C: 民國年 YYYMMDD (7碼) -> 轉西元
        if date_str.isdigit() and len(date_str) == 7:
            try:
                roc_year = int(date_str[:3])
                ad_year = roc_year + 1911
                return f"{ad_year}-{date_str[3:5]}-{date_str[5:7]}"
            except ValueError:
                return None

        return None